In [1]:
from open_clip.jt_ViT_RelClassifier import ViT_RelClassifier
from jt_training import get_dataloader, train_one_epoch, evaluate, get_free_gpu

def load_model(clip_model_type, clip_pretrained_dataset, n_rel_classes, n_obj_classes):
    model = ViT_RelClassifier(n_rel_classes, n_obj_classes, clip_model_type, clip_pretrained_dataset)
    prepocess_function = model.preprocess
    device = get_free_gpu()
    model.to(device)
    return model, prepocess_function, device

In [2]:
import torch
clip_model_type = 'ViT-B/32'
clip_pretrained_dataset = 'laion400m_e32'
image_dir = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/VG/"
metadata_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"
model, prepocess_function, device = load_model(clip_model_type, clip_pretrained_dataset, 100, 200)
model.load_state_dict(torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-04-16/vision_transformer_0/checkpoints/best_rel_model.pt"))
# dataloader_train, dataloader_val = get_dataloader(prepocess_function,metadata_path,image_dir, testing_only=False)

<All keys matched successfully>

In [5]:
### load the first batch and show the predictions
import torch
import matplotlib.pyplot as plt
model.eval()
for (inputs, bounding_boxes, lrels, lobj1s, lobj2s) in dataloader_val:
    rel, obj_1, obj_2 = model(inputs.to(device), bounding_boxes.to(device))
    _, rel_preds = torch.max(rel, 1)
    _, obj_1_preds = torch.max(obj_1, 1)
    _, obj_2_preds = torch.max(obj_2, 1)
    print("relationship predictions:", rel_preds)
    print("object 1 predictions:", obj_1_preds)
    print("object 2 predictions:", obj_2_preds)
    break

relationship predictions: tensor([ 2,  0,  0,  4,  1,  0,  4,  0,  0,  1,  1,  3,  0,  0, 13,  1,  1,  6,
         0,  2,  4,  0,  4,  0,  0,  4,  0,  2,  2,  9,  4,  2, 16,  4,  0,  4,
         0,  0,  0,  3,  2,  0,  0,  2,  0,  0,  0,  2,  0,  4,  3,  0,  0,  2,
         4,  0,  0,  0,  0,  4,  0,  4,  0,  0], device='cuda:0')
object 1 predictions: tensor([  4,  22,  13,   7,  64,  42,   0,  81,  83,  62, 163,  30,  37,  45,
         13,   0,  54,   3,  28,   0,   7,  92,   0,  45,  11,   1,  47,  47,
         85,  13,   0,   1,   9,  52,  11,   1,  21, 120,  48,  21,   1,  15,
         51,  93,  72,  28, 104, 124,   2,  42,  30,  77, 128,  47,  42,   3,
         19,  21,   5,   0,  76,  52,   2,   2], device='cuda:0')
object 2 predictions: tensor([ 99,  37,   8,  65,  24, 124,   5,  17,  31,  79,   9,   0,  37,   6,
         63,  18,  30,  27,   4, 124,  40,   6,   5,   4,  27,  40,  16,  16,
         16, 104,   5, 163, 130,  44,  10,  29,   0,  12,  60,  73, 171,  13,
          8,

the model doesn't always predict a constant which is very good.

In [3]:
from jt_training import get_realistic_graphs_dataset_ViT

In [8]:
import random
import PIL
dataset_orig, dataset_adv, list_to_iterate = get_realistic_graphs_dataset_ViT(prepocess_function, image_dir)
correct = 0
total = 0
for d in list_to_iterate:
    original_graph = d['original_graph']
    adv_graph = d['adv_graph']
    changed_edge = d['changed_edge']
    image_id = original_graph.image_id
    inputs, bounding_boxes, rel_label, obj1_label, obj2_label = dataset_orig.getitem_from_id_edge(image_id, changed_edge)
    inputs2,_, rel_label_adv, obj1_label_adv, obj2_label_adv = dataset_adv.getitem_from_id_edge(image_id, changed_edge)
    assert (inputs == inputs2).all()
    # print("original relationship label:", rel_label)
    # print("adversarial relationship label:", rel_label_adv)
    rel, obj_1, obj_2 = model(inputs.unsqueeze(0).to(device), bounding_boxes.unsqueeze(0).to(device))
    rel_adv, obj_1_adv, obj_2_adv = model(inputs.unsqueeze(0).to(device), bounding_boxes.unsqueeze(0).to(device))

    original_rel_confidence = rel[0][rel_label].item()
    adversarial_rel_confidence = rel_adv[0][rel_label_adv].item()
    if original_rel_confidence > adversarial_rel_confidence:
        correct += 1
    total += 1
    # print("original relationship confidence:", original_rel_confidence)
    # print("adversarial relationship confidence:", adversarial_rel_confidence)
    # break
print("accuracy:", correct/total)

Filtered relationships loaded from file
Filtered objects loaded from file
Filtered relationships loaded from file
Filtered objects loaded from file
accuracy: 0.8446601941747572
